In [ ]:
from ml_pipe_space_model_batches import insert_data
dataset_df  = insert_data()

In [ ]:
import os 
from ml_pipe_data_collection import DataCollection

room_id  ="room_1"

def preprocessing_function(dataset,room_id):

    """
    Args: Require dataframe as input,
    Room id to create folder for that room and save data for specific room.

    return:Numpy file at specific folder

      """
    data_collection = DataCollection(room_id, dataset, nan_interpolation_gap_limit=36, n_sequence=144)
    data_collection.prepare_for_data_batches()

    # this folder structure is created for jupyter notebbook
    save_folder = os.path.join(room_id, "space_model_dataset")
    if not os.path.exists(save_folder):
    # create the directory
        os.makedirs(save_folder)
    data_collection.create_data_batches(save_folder=save_folder)

    save_folder = os.path.join(room_id, "space_models", "BMS_data")
    if not os.path.exists(save_folder):
    # create the directory
        os.makedirs(save_folder)
    data_collection.save_building_data_collection_dict(save_folder=save_folder)

preprocessing_function(dataset_df,room_id)

In [ ]:
from tfx.components import CsvExampleGen
from tfx.components import StatisticsGen
from tfx.components import SchemaGen
from tfx.components import ExampleValidator
from tfx.components import Transform
from tfx.components import Trainer
from tfx.components import Evaluator
from tfx.components import ModelValidator
from tfx.components import Pusher
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

import tensorflow_data_validation as tfdv
from tensorflow_data_validation.utils import slicing_util
from tensorflow_metadata.proto.v0.statistics_pb2 import DatasetFeatureStatisticsList, DatasetFeatureStatistics

In [ ]:
import tensorflow as tf
import tensorflow_data_validation as tfdv
import numpy as np

# Load the .npz file
data = np.load("room_1\\space_model_dataset\\room_1_training.npz")

x = data[data.files[0]]
y = data[data.files[1]]

x = x[:,:-1]
y= y[:,1:]-y[:,:-1]

# Assume that each row of `features` corresponds to the same row as `labels`.
assert x.shape[0] == y.shape[0]

dataset = tf.data.Dataset.from_tensor_slices((x, y))



In [ ]:
# Generate statistics from the dataset
stats = tfdv.generate_statistics_from_tfrecord(dataset)
tfdv.visualize_statistics(stats)

# Define the schema for the dataset
schema = tfdv.infer_schema(statistics=stats)

# Print the schema and the anomalies
print("Schema:")
tfdv.display_schema(schema)

In [ ]:
Data_stats = tfdv.generate_statistics_from_dataframe(df)
tfdv.visualize_statistics(Data_stats)

In [ ]:
schema = tfdv.infer_schema(statistics=Data_stats)
tfdv.display_schema(schema)

In [ ]:
# Define the input and output locations for the pipeline
input_data = "'temp_file_room_1.csv'"
output_data = "./"

# Define the pipeline
example_gen = CsvExampleGen(input_base=input_data)
statistics_gen = StatisticsGen(input_data=example_gen.outputs['examples'])
schema_gen = SchemaGen(input_data=statistics_gen.outputs['output'])
example_validator = ExampleValidator(stats=statistics_gen.outputs['output'], schema=schema_gen.outputs['schema'])
transform = Transform(input_data=example_gen.outputs['examples'], schema=schema_gen.outputs['schema'])
trainer = Trainer(input_data=transform.outputs['transformed_examples'], schema=schema_gen.outputs['schema'])
evaluator = Evaluator(input_data=transform.outputs['transformed_examples'], schema=schema_gen.outputs['schema'], model=trainer.outputs['model'])
model_validator = ModelValidator(model=trainer.outputs['model'], examples=example_gen.outputs['examples'], schema=schema_gen.outputs['schema'])
pusher = Pusher(input_data=trainer.outputs['model'])

# Define the pipeline
pipeline_name = 'my_pipeline'
pipeline_root = output_data
pipeline_args = {}
pipeline_args['pipeline_name'] = pipeline_name
pipeline_args['pipeline_root'] = pipeline_root
pipeline_args['input_data'] = input_data
pipeline_args['output_data'] = output_data
pipeline_args['components'] = [example_gen, statistics_gen,schema_gen, example_validator, transform, trainer, evaluator, model_validator, pusher]

# Create and run the pipeline
p = pipeline.Pipeline(**pipeline_args)
